In [1]:
import os, sys, shutil, glob
import numpy as np
import pandas as pd
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
import matplotlib.pyplot as plt

In [2]:
# import modules
import sklearn
from sklearn.metrics import make_scorer, mean_squared_log_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split as tts
from sklearn import ensemble
from sklearn.preprocessing import OneHotEncoder as OHE
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import scipy as sp
import math

## DATA Load

In [3]:
base_dir = os.getcwd()
train_dir = os.path.join(base_dir, 'train/train.csv')

In [4]:
base_df = pd.read_csv(train_dir)
base_df.head()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,1.5,69.08,-12,0.0
1,0,0,30,0,0,1.5,69.06,-12,0.0
2,0,1,0,0,0,1.6,71.78,-12,0.0
3,0,1,30,0,0,1.6,71.75,-12,0.0
4,0,2,0,0,0,1.6,75.20,-12,0.0


## PREPROCESS

In [5]:
base_df['Time'] = base_df['Hour']*60 + base_df['Minute']

dataset = base_df[['Day', 'Time', 'DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET']]

In [6]:
dataset

,Day,Time,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,1.5,69.08,-12,0.0
1,0,30,0,0,1.5,69.06,-12,0.0
2,0,60,0,0,1.6,71.78,-12,0.0
3,0,90,0,0,1.6,71.75,-12,0.0
4,0,120,0,0,1.6,75.20,-12,0.0
...,...,...,...,...,...,...,...,...
52555,1094,1290,0,0,2.4,70.70,-4,0.0
52556,1094,1320,0,0,2.4,66.79,-4,0.0
52557,1094,1350,0,0,2.2,66.78,-4,0.0
52558,1094,1380,0,0,2.1,67.72,-4,0.0


In [108]:
# Scaler -- standard or minmax? 0~1?
scaleset = dataset.drop('Day', axis=1).values
scaler = MinMaxScaler()
scaler.fit(scaleset)

MinMaxScaler()

In [109]:
all_blocks = []
for i in range(0, 1094-3):
    temp_block = dataset.loc[(dataset.Day >= i) & (dataset.Day <= i + 4)]
    temp_block = temp_block.drop('Day', axis = 1)
    all_blocks.append(temp_block)

In [116]:
# train/val blocks
import random

def pick_one_batch(train_num, val_num, base_block):
    
    train_num = train_num
    val_num = val_num
    all_blocks = base_block

    train_batch_idx = [random.choice(range(len(all_blocks))) for i in range(train_num)]
    train_batch = [all_blocks[i] for i in train_batch_idx]

    val_batch_cand = [k for i, k in enumerate(all_blocks) if i not in train_batch_idx]
    val_batch = [random.choice(val_batch_cand) for i in range(val_num)]

    train_x = torch.tensor(np.array([scaler.transform(batch.values)[:144, :] for batch in train_batch])).float()
    train_x = train_x.permute(0, 2, 1)
    train_y = torch.tensor(np.array([batch.values[144:, -1] for batch in train_batch])).float()

    val_x = torch.tensor(np.array([scaler.transform(batch.values)[:144, :] for batch in val_batch])).float()
    val_x = val_x.permute(0, 2, 1)
    val_y = torch.tensor(np.array([batch.values[144:, -1] for batch in val_batch])).float()
    
    return train_x, train_y, val_x, val_y

In [237]:
class DaconCNN(nn.Module):
    def __init__(self, feature_dim):
        super(DaconCNN, self).__init__()
        
        # block1
        self.block1 = nn.Sequential(
            nn.Conv1d(in_channels=feature_dim, out_channels=32, kernel_size=6),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=6),
            nn.BatchNorm1d(64),
            nn.ReLU(),
        )
        
        self.block2 = nn.Sequential(
            nn.Linear(64*134, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            
            nn.Linear(1024, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            
            nn.Linear(256, 96),
        )
        
    def forward(self, x):
        
        out = self.block1(x)
        #print(out.size())
        batch_size = out.size()[0]
        
        out = out.view(batch_size, -1)
        out = self.block2(out)
        
        out = torch.max(torch.zeros_like(out), out)
        return out

In [260]:
def QuantileLoss(qs, pred, gt):
    #qs = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
    qs = qs if isinstance(qs, list) else [qs]
    
    sum_loss = 0
    for i, q in enumerate(qs):
        loss = pred - gt
        loss = torch.max(q*loss, (q-1)*loss)
        sum_loss += torch.mean(loss)
        
    fin_loss = sum_loss/len(qs)# + 0.05*symloss/4
    return fin_loss

In [261]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def trainCNN(epochs, base_block, qs):
    
    train_num = 32
    val_num = 16
    qs = qs
    
    model = DaconCNN(feature_dim=7).float()
    
    learning_rate = 0.005
    optimizer = optim.Adam(model.parameters(), lr = learning_rate)
    
    decay_rate = 0.998
    lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer,
                                                         gamma=decay_rate)
    
    train_loss_sum = 0.0
    val_loss_sum = 0.0
    
    for epoch in range(epochs):
        
        model.train()
        
        for i in range(30):
            
            train_x, train_y, val_x, val_y = pick_one_batch(train_num, val_num, base_block)
            optimizer.zero_grad()
            
            train_pred = model(train_x)
            train_loss = QuantileLoss(qs, train_pred, train_y)
            
            train_loss.backward()
            optimizer.step()
            
            train_loss_sum += train_loss
            
        with torch.no_grad():
            val_pred = model(val_x)
            val_loss = QuantileLoss(qs, val_pred, val_y)
            val_loss_sum += val_loss
                
        print(f"Epoch: {epoch+1} | Loss: {train_loss_sum.item()/30:.4f} | Val Loss: {val_loss.item():.4f}")
        
        train_loss_sum, val_loss_sum = 0.0, 0.0
        
        lr_scheduler.step()
        
    return model
                

In [262]:
epochs = 200

model_fit_4 = trainCNN(epochs, all_blocks, 0.9)

torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([32, 96])
torch.Size([32, 96]) torch.Size([3

KeyboardInterrupt: 

In [140]:
qlist = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
epochs = 40

all_models = []
for q in qlist:
    temp_model = trainCNN(epochs, all_blocks, q)
    all_models.append(temp_model)

Epoch: 1 | Loss: 1.4128 | Val Loss: 1.2064
Epoch: 2 | Loss: 1.2847 | Val Loss: 1.2453
Epoch: 3 | Loss: 1.1862 | Val Loss: 1.4324
Epoch: 4 | Loss: 1.0724 | Val Loss: 1.2983
Epoch: 5 | Loss: 0.9590 | Val Loss: 1.0381
Epoch: 6 | Loss: 0.8556 | Val Loss: 0.8214
Epoch: 7 | Loss: 0.7700 | Val Loss: 0.8628
Epoch: 8 | Loss: 0.6728 | Val Loss: 0.6803
Epoch: 9 | Loss: 0.6119 | Val Loss: 0.6925
Epoch: 10 | Loss: 0.5576 | Val Loss: 0.7278
Epoch: 11 | Loss: 0.5197 | Val Loss: 0.6069
Epoch: 12 | Loss: 0.4956 | Val Loss: 0.7399
Epoch: 13 | Loss: 0.4701 | Val Loss: 0.5235
Epoch: 14 | Loss: 0.4416 | Val Loss: 0.4930
Epoch: 15 | Loss: 0.4332 | Val Loss: 0.5391
Epoch: 16 | Loss: 0.4135 | Val Loss: 0.4071
Epoch: 17 | Loss: 0.4082 | Val Loss: 0.4065
Epoch: 18 | Loss: 0.3856 | Val Loss: 0.5517
Epoch: 19 | Loss: 0.3889 | Val Loss: 0.4522
Epoch: 20 | Loss: 0.3656 | Val Loss: 0.4487
Epoch: 21 | Loss: 0.3564 | Val Loss: 0.5358
Epoch: 22 | Loss: 0.3457 | Val Loss: 0.4015
Epoch: 23 | Loss: 0.3447 | Val Loss: 0.63

In [259]:
model_fit_3.eval()
model_fit_3(X_test_scaled.unsqueeze(0).permute(0, 2, 1))

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000, 10.3519, 18.5233, 25.7514, 35.1429,
         45.6047, 53.3597, 62.5164, 67.6771, 73.8580, 64.1007, 64.0721, 69.7000,
         56.7913, 39.7921, 42.3076, 42.9325, 43.5416, 36.1186, 24.8013, 21.5130,
         29.4884, 22.9956, 11.6964,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  5.8725, 10.9380, 16.3496, 24.0004, 28.6765,
         41.3924, 49.4036, 54.1940, 64.8875, 70.2591, 65.3948, 51.0579, 42.9892,
         34.4777, 30.9703, 18.5401, 31.7124, 37.8042, 28.2661, 25.8330, 14.9887,
         12.1981, 12.8575, 21.4388,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],
       grad_fn=<MaximumBac

In [257]:
model_fit_2.eval()
model_fit_2(X_test_scaled.unsqueeze(0).permute(0, 2, 1))

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  2.7911, 10.2532, 21.8295, 29.3181, 39.5143, 49.3631,
         59.0885, 67.4031, 75.5168, 82.6882, 88.1655, 91.8142, 94.7293, 98.3151,
         99.6122, 98.5456, 96.4506, 94.9549, 90.8239, 84.3968, 77.3709, 69.1803,
         60.0362, 51.8653, 41.1143, 31.0865, 20.9020, 12.0363,  4.1673,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000, 11.7293, 21.1347, 30.1212, 39.4605, 49.0026,
         58.0772, 66.6953, 74.3474, 80.5856, 86.0345, 88.9672, 90.9638, 93.7306,
         93.9360, 92.3277, 92.0909, 87.9208, 83.1364, 78.2522, 71.6307, 63.4750,
         54.8168, 47.5863, 38.9608, 29.7340, 20.2401, 12.1814,  5.1140,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],
       grad_fn=<MaximumBac

In [254]:
model_fit_2.eval()
model_fit_2(X_test_scaled.unsqueeze(0).permute(0, 2, 1))

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  6.7767, 13.6113, 20.9538, 28.2056, 37.3899,
         49.4168, 57.1391, 64.8364, 69.0400, 64.1241, 56.3657, 40.4668, 42.2490,
         57.6851, 78.4175, 70.3783, 61.8727, 56.6361, 55.5211, 51.8121, 40.8567,
         41.8397, 36.8074, 30.5164, 23.1708, 13.7557,  6.4310,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  7.4615, 14.6673, 23.4963, 31.9420, 40.0800,
         51.5010, 61.4924, 70.6849, 82.1008, 82.9252, 86.2703, 84.5968, 87.4733,
         76.5311, 76.9198, 74.5990, 60.1821, 62.1870, 56.3080, 48.8988, 31.9921,
         28.9631, 23.8986, 19.0518, 16.7852, 10.0963,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],
       grad_fn=<MaximumBac

In [247]:
traintp, trainytp, _, _ = pick_one_batch(1, 1, all_blocks)

In [251]:
traintp

tensor([[[0.0000, 0.0213, 0.0426,  ..., 0.9574, 0.9787, 1.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [1.0780, 1.1873, 1.1675,  ..., 0.9916, 1.0127, 1.0120],
         [1.8000, 1.8000, 1.8000,  ..., 1.8667, 1.8667, 1.8000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]])

In [191]:
aa = all_models[0]
aa(X_test_scaled.unsqueeze(0).permute(0, 2, 1))

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 14.8106, 24.0457, 32.9695,
         37.2642, 44.1790, 53.7278, 66.0208, 73.4586, 48.7767, 62.5999, 55.5483,
         37.7692, 30.8488, 36.1534, 57.2513, 59.9888, 39.2756, 22.5861, 22.9690,
         17.2192, 21.8272, 16.3000, 11.9697,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  7.7782, 11.2605, 18.2231, 25.9930,
         35.5062, 42.3293, 50.7710, 73.3819, 79.6653, 82.6055, 58.2425, 51.9659,
         41.1143, 39.7554, 17.5615, 37.9111, 34.1295, 55.9844, 40.5281, 18.4217,
         25.1828, 17.8163, 17.2617, 13.4249,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],
       grad_fn=<MaximumBac

In [135]:
test_dir = os.path.join(base_dir, 'test')
lists = os.listdir(test_dir)

In [185]:
df_test = []
result_pd = pd.DataFrame()

for i in range(81):
    file_path = os.path.join(test_dir, str(i)+'.csv')
    temp = pd.read_csv(file_path)
    temp['Time'] = temp['Hour']*60 + temp['Minute']
    temp = temp.loc[(temp.Day >= 4) & (temp.Day <= 6), :][['Time', 'DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET']]
    
    X_test_scaled = torch.tensor(scaler.transform(temp.values)).float()
    
    temp_result_list = []
    for m in range(9):
        temp_model = all_models[m]
        temp_model.eval()
        temp_result = temp_model(X_test_scaled.unsqueeze(0).permute(0,2,1)).detach().to('cpu').numpy().squeeze(0)
        temp_result_list.append(pd.DataFrame(temp_result))
    
    result_pd = result_pd.append(pd.concat(temp_result_list, axis=1))
    
        
        

In [186]:
result_pd

,0,0,0,0,0,0,0,0,0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
91,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [187]:
result_pd.iloc[:48, :]

,0,0,0,0,0,0,0,0,0
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [136]:
X_test = pd.concat(df_test)
X_test.shape

(144, 7)

In [137]:
X_test_scaled = torch.tensor(scaler.transform(X_test.values)).float()

In [138]:
model_fit_1.eval()
model_fit_1(X_test_scaled.unsqueeze(0).permute(0,2,1))

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.3822,
          4.9136, 12.8338, 15.7965, 21.9989, 24.5731, 25.1446, 33.5316, 39.8747,
         41.4034, 45.7704, 34.9744, 30.8550, 30.5020, 25.3337, 27.3108, 14.1588,
          7.3356,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.2800,
          6.5677, 14.9884, 20.0625, 23.7999, 25.7666, 34.4319, 49.9423, 47.7675,
         41.2324, 44.8759, 37.3615, 35.7089, 41.2612, 34.9380, 26.2835, 19.5008,
         10.4811,  0.1983,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],
       grad_fn=<MaximumBac